# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle
from config import *

In [3]:
features = ['Was_home',
 'Rating_difference',
 'Value',
 'Transfers_balance',
 'Avg_FPL_points',
 'Min_2',
 'Team_CS_2',
 'Player_GC_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'Saves_2',
 'FPL_points_2',
 'Min_4',
 'Team_CS_4',
 'Player_GC_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'Saves_4',
 'FPL_points_4',
 'Min_30',
 'Team_CS_30',
 'Player_GC_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'Saves_30',
 'FPL_points_30']

info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]

In [4]:
len(features)

29

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning: Columns (2,3,6,8,36,43,44,45,59,84,86,147,208) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df = df[(df["Season"] == CURRENT_SEASON) & (df["FPL_pos"] == "GK")]

In [7]:
X_test = df[features]

In [8]:
df.columns

Index(['Unnamed: 0', 'Date', 'Day', 'Venue', 'Team', 'Opponent', 'Name',
       'Start', 'Pos', 'Min',
       ...
       'FPL_xGC_30', 'Opp_rating_30', 'Team_rating_30', 'Min_points_30',
       'Team_result_30', 'DEF', 'FWD', 'MID', 'Finished', 'Price'],
      dtype='object', length=274)

In [9]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Value', 'Transfers_balance',
       'Avg_FPL_points', 'Min_2', 'Team_CS_2', 'Player_GC_2', 'Team_score_2',
       'Opp_score_2', 'Team_result_2', 'Saves_2', 'FPL_points_2', 'Min_4',
       'Team_CS_4', 'Player_GC_4', 'Team_score_4', 'Opp_score_4',
       'Team_result_4', 'Saves_4', 'FPL_points_4', 'Min_30', 'Team_CS_30',
       'Player_GC_30', 'Team_score_30', 'Opp_score_30', 'Team_result_30',
       'Saves_30', 'FPL_points_30'],
      dtype='object')

In [10]:
def new_predict_gk(X):
    XGB_COMPONENT = 0.3
    OVERALL_COMPONENT = 0.67
    FORM_COMPONENT = 0.2
    
    model_pred = np.array(model.predict2(X))
    overall_pred = np.array([row["Avg_FPL_points"] for i, row in X.iterrows()])
    form_pred = np.array([row["FPL_points_4"] for i, row in X.iterrows()])
    
    return np.add( model_pred * XGB_COMPONENT, overall_pred * OVERALL_COMPONENT, form_pred * FORM_COMPONENT )

file = open("models/GBR_gk.pkl",'rb')
model = pickle.load(file)

In [11]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [12]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [13]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [14]:
preds = get_predictions(model, df, X_test)

1717 (1717, 274)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [16]:
# next gameweek
preds[preds["GW"] == 3].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
173,André Onana,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,GK,50.0,7.042664
101,Alisson Ramses Becker,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,GK,55.0,4.393918
1216,Nick Pope,3.0,2023-24,Newcastle Utd,Liverpool,1.0,1887.064819,1943.976074,GK,55.0,4.008226
461,David Raya Martin,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,49.0,3.985932
1646,Wes Foderingham,3.0,2023-24,Sheffield United,Manchester City,1.0,1637.473755,2080.850586,GK,45.0,3.813010
533,Ederson Santana de Moraes,3.0,2023-24,Manchester City,Sheffield United,0.0,2080.850586,1637.473755,GK,55.0,3.732560
712,Hugo Lloris,3.0,2023-24,Tottenham,Bournemouth,0.0,1817.656738,1662.536621,GK,50.0,3.660901
29,Aaron Ramsdale,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,GK,50.0,3.658999
568,Emiliano Martínez Romero,3.0,2023-24,Aston Villa,Burnley,0.0,1816.929932,1722.802979,GK,50.0,3.534537
820,José Malheiro de Sá,3.0,2023-24,Wolves,Everton,0.0,1715.631348,1699.505859,GK,50.0,3.472982


In [17]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
190,André Onana,20.0,2023-24,Manchester Utd,Nott'ham Forest,0.0,1877.037842,1671.868774,GK,50.0,7.042664
203,André Onana,33.0,2023-24,Manchester Utd,Bournemouth,0.0,1877.037842,1662.536621,GK,50.0,7.042664
175,André Onana,5.0,2023-24,Manchester Utd,Brighton,1.0,1877.037842,1832.364258,GK,50.0,7.042664
176,André Onana,6.0,2023-24,Manchester Utd,Burnley,0.0,1877.037842,1722.802979,GK,50.0,7.042664
177,André Onana,7.0,2023-24,Manchester Utd,Crystal Palace,1.0,1877.037842,1763.994995,GK,50.0,7.042664
178,André Onana,8.0,2023-24,Manchester Utd,Brentford,1.0,1877.037842,1822.547119,GK,50.0,7.042664
179,André Onana,9.0,2023-24,Manchester Utd,Sheffield United,0.0,1877.037842,1637.473755,GK,50.0,7.042664
181,André Onana,11.0,2023-24,Manchester Utd,Fulham,0.0,1877.037842,1745.365723,GK,50.0,7.042664
182,André Onana,12.0,2023-24,Manchester Utd,Luton,1.0,1877.037842,1602.327637,GK,50.0,7.042664
183,André Onana,13.0,2023-24,Manchester Utd,Everton,0.0,1877.037842,1699.505859,GK,50.0,7.042664


In [18]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
André Onana,255.209582
Alisson Ramses Becker,161.615735
Nick Pope,142.870684
David Raya Martin,142.181040
Wes Foderingham,142.163791
Aaron Ramsdale,135.486704
Ederson Santana de Moraes,134.083294
Emiliano Martínez Romero,128.122819
Hugo Lloris,127.052841


# Saving to file

In [19]:
preds.to_csv("predictions/preds_next_season_gk.csv", index=False)

# Importance

In [20]:
# import shap
# explainer = shap.Explainer(model.predict, X_test[9000:10000])
# shap_values = explainer(X_test[9000:10000])

In [21]:
# shap.plots.bar(shap_values, max_display=15)

In [22]:
# # Salah vs Nottingham Forest
# shap.plots.waterfall(shap_values[502])

In [23]:
# shap.plots.waterfall(shap_values[470])